### Description

In this task you will work with fault injection before the last Sbox operation. 

This fault model is used in practice. A fault shall be injected during algorithm run, so, this fault can be considered as dynamic.

Any fault injection method, i.e, a glitch, laser, EM, etc., can achieve this kind of effect.


### Fault model

This is 1-bit fault, i.e., a random bit before Sbox operation is swapped.

s' = s ^ error_bit

where s is a byte of the AES128 State just before the last Sbox
error_bit = [1 2 4 8 16 32 64 128]

### Task:

Your task is to find a Master key (round key 0) embedded into ../aes128 binary. 

The master key is in the form of HEIG{XXXXXXXXX}, where X is a ASCII printable symbol.

### Fault illustration

<img src="support/fault.png">

In [121]:
import numpy as np
import binascii
import string
import random
import socket

HOST = 'iict-mv330-sfa'
PORT = 5002

from tqdm import auto as tqdm
import sca_training

def binary_aes128_encrypt_fault(plaintext, fault_ind, verbose=False):
    import subprocess
    if all(c in string.hexdigits for c in plaintext) and len(plaintext) == 32:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.connect((HOST, PORT))
            s.recv(1024)
            s.sendall(plaintext.encode())
            s.sendall(b'\n')

            output = s.recv(1024)
    
    c_ctext = binascii.unhexlify(output[10:42])
    f_ctext = binascii.unhexlify(output[53:85])
    xor =  int.from_bytes(c_ctext, byteorder = 'big') ^ int.from_bytes(f_ctext, byteorder = 'big')
    
    if (verbose):
        print('Full output:', output.strip())
        print('Correct ciphertext: {}'.format(c_ctext.hex()) )
        print('Faulted ciphertext: {}'.format(f_ctext.hex()) )
        print('Difference (XOR):   {:032X}'.format(xor) )
    
    return output, c_ctext, f_ctext

In [106]:
ptext = "000102030405060708090a0b0c0d0e0F"
output, c_ctext, f_ctext = binary_aes128_encrypt_fault(ptext, 15, verbose=True)

Full output: b'[correct] C0515E087F4B134DDC3CFB73C1DEF395\n[faulted] C0515E087F51134DDC3CFB73C1DEF395'
Correct ciphertext: c0515e087f4b134ddc3cfb73c1def395
Faulted ciphertext: c0515e087f51134ddc3cfb73c1def395
Difference (XOR):   00000000001A00000000000000000000


In [130]:
from collections import Counter
e = [0x1, 0x2, 0x4, 0x8, 0x10, 0x20, 0x40, 0x80] # 1 byte error

SAMPLES = 100

def attack():
  key_cand = np.zeros((16, 256)).astype(int)
  for s in tqdm.trange(SAMPLES):
    plain = ''.join(random.choice(string.hexdigits) for i in range(32))
    _, ctext, ftext = binary_aes128_encrypt_fault(ptext, 15)
    for j in range(256):
      for i in range(16):
        e_cand = sca_training.invSbox[ctext[i] ^ j] ^ sca_training.invSbox[ftext[i] ^ j]
        if e_cand in e:
          key_cand[i][j] += 1
          break
  return key_cand
key = attack()
round_key = np.zeros(16).astype(int)
for i in range(16):
  # print(np.argmax(key[i]))
  round_key[i] = np.argmax(key[i])
# print(key)
print(round_key)

  0%|          | 0/100 [00:00<?, ?it/s]

[  2  40 143 198 223 243 161  86 217  70 189 249 162 245 222 236]


In [131]:
#Get master key using the provided binary
key_schedule = sca_training.inverse_key_expansion(round_key)
print(key_schedule.shape)

print('Master key in hex:', key_schedule[0,0,:])

print('Master key in ASCII:', binascii.unhexlify(''.join('{:02x}'.format(c) for c in key_schedule[0,0,:])))

(1, 11, 16)
Master key in hex: [ 72  69  73  71 123  49  48 114 111 117 110 100  98 105 116 125]
Master key in ASCII: b'HEIG{10roundbit}'
